In [ ]:
import labelbox
import os
LB_API_KEY = ''
PROJECT_ID = ''
client = labelbox.Client(api_key = LB_API_KEY)
project = client.get_project(PROJECT_ID)
labels = project.export_v2(params={
	"data_row_details": False,
	"metadata_fields": False,
	"attachments": False,
	"project_details": False,
	"performance_details": False,
	"label_details": False,
	"interpolated_frames": False
  })
labels.wait_till_done()
if labels.errors:
	print(labels.errors)
export_json = labels.result
type(export_json)        
	# os.system("echo " + export_json + "\n >> data.json")
	

In [ ]:
import os
os.system("rm test.txt && touch test.txt")
t_file = open("test.txt", "a")

def reset(all=False):
    if all:
        os.system("rm -rf /home/th3will/dev/OTUMEDAI/training_data/yolo")
    else:
        os.system("cd /home/th3will/dev/OTUMEDAI/training_data/yolo/images/train && rm *")
        os.system("cd /home/th3will/dev/OTUMEDAI/training_data/yolo/labels/train && rm *")

In [ ]:
import cv2
from pathlib import Path

def create_train_infrastructure():
    # str_dir = input("Enter the path to the training folder to export the yolo files to: ")
    str_dir = "/home/th3will/dev/OTUMEDAI/training_data"
    end_dir = Path(str_dir)
    # make the yolo folder and subfolders
    os.makedirs(os.path.join(end_dir, "yolo", "images", "train"))
    os.makedirs(os.path.join(end_dir, "yolo", "images", "val"))
    os.makedirs(os.path.join(end_dir, "yolo", "labels", "train"))
    os.makedirs(os.path.join(end_dir, "yolo", "labels", "val"))
    return os.path.join(end_dir, "yolo")

def setup_video_labels(vid_name, path_to_yolo):
    vid = cv2.VideoCapture(vid_name)
    success,image = vid.read()
    prefix = vid_name.split(".")[0]
    frame_count = 0
    write_path = Path(path_to_yolo)
    # print(image)
    while success:
        cv2.imwrite(os.path.join(write_path,'images/train',(prefix+"%d.jpg" % frame_count)), image) # save frame as JPEG file 
        f = open(os.path.join(write_path,'labels/train',(prefix+"%d.txt" % frame_count)),'x')     
        # print(os.path.join(write_path,'images',(prefix+"%d.jpg" % frame_count)))
        success,image = vid.read()
        frame_count += 1
        

In [ ]:
import cv2
#To-do: add multi-class support
reset(all=True)
root = create_train_infrastructure()
for dict in export_json:
    height = dict["media_attributes"]["height"]
    width = dict["media_attributes"]["width"]
    vid_name = dict["data_row"]["external_id"]
    # print(vid_name)
    setup_video_labels(vid_name, root)
    prefix = vid_name.split(".")[0]
    # print(vid_name)
    vid = cv2.VideoCapture(vid_name)
    success,image = vid.read()
    # print(image)
    # t_file.write(str(image.tolist()))
    # print(height, width)
    for i in dict["projects"]:
        # print(dict["projects"][i]["labels"])
        label_list = dict["projects"][i]["labels"]
        for j in label_list:
            # print(j["annotations"])
            for frame_num in j["annotations"]["frames"]:
                f_path = os.path.join(root, "labels", "train", prefix + frame_num + ".txt")
                #frame_num is the frame number
                t_file.write(str(j["annotations"]["frames"][frame_num]) + "\n")
                print(frame_num)
                print(j["annotations"]["frames"])
                objects = j["annotations"]["frames"][frame_num]["objects"]
                #print(str(objects))
                for things in objects:
                    # print(objects[things]["name"])
                    class_name = objects[things]["name"]
                    t_file.write(str(objects[things]) + "\n")
                    bbox = objects[things]["bounding_box"]
                    # print(bbox)
                    #bbox processing
                    #bbox gives things in terms of top left corner, width, height
                    x = bbox["left"]/width if bbox["left"] > 0 else 0
                    y = bbox["top"]/height if bbox["top"] > 0 else 0
                    x_width = bbox["width"]/width if x + bbox["width"] < width else (width - x)/width
                    y_height = bbox["height"]/height if y + bbox["height"] < height else (height - y)/height
                    # this gives reasonable values for x, y, x_width, y_height but they need to be changed into yolo format
                    # x_center, y_center, x_width, y_height
                    x = x + x_width/2
                    y = y + y_height/2
                    # print(x, y, x_width, y_height)
                    # TODO create robust class selection system
                    if class_name == "kidney stone":
                        with open(f_path, "a") as f:
                            f.write("0 " + str(x) + " " + str(y) + " " + str(x_width) + " " + str(y_height) + "\n")
                    
                    
        
        

In [ ]:
#Split training and testing data
import random, shutil, os
random.seed()

path_to_data = '/home/th3will/dev/OTUMEDAI/training_data/yolo' 
list_of_pre = []
#20% of the data is used for val
for file in os.listdir(os.path.join(path_to_data, "images", "train")):
    list_of_pre.append(file.split('.')[0])
size_of_list = len(list_of_pre)
while len(list_of_pre) > size_of_list*0.8:
    pre_frame = list_of_pre.pop(random.randint(0, len(list_of_pre)-1)) 
    shutil.move(os.path.join(path_to_data, "images", "train", pre_frame+".jpg"), os.path.join(path_to_data, "images", "val", pre_frame+".jpg"))
    shutil.move(os.path.join(path_to_data, "labels", "train", pre_frame+".txt"), os.path.join(path_to_data, "labels", "val", pre_frame+".txt"))



Sus stuff below

In [ ]:
#deletes all frames/frame data for test2 after frame 302
#I got lazy in the annotation
import os
from pathlib import Path
label_path = Path("/home/th3will/dev/OTUMEDAI/training_data/yolo/labels/train")
video_path = Path("/home/th3will/dev/OTUMEDAI/training_data/yolo/images/train")
v_dirlist = os.listdir(video_path)
l_dirlist = os.listdir(label_path)
for v_dir in v_dirlist:
    prefix = v_dir.split("_c")[0]
    frame_num = v_dir.split("_c")[1].split(".")[0]
    if(prefix == "test2") and (int(frame_num) > 302 ):
        if os.path.exists(os.path.join(video_path, v_dir)):
            os.remove(os.path.join(video_path, v_dir))
for l_dir in l_dirlist:
    prefix = l_dir.split("_c")[0]
    frame_num = l_dir.split("_c")[1].split(".")[0]
    if(prefix == "test2") and (int(frame_num) > 302 ):
        if os.path.exists(os.path.join(label_path, l_dir)):
            os.remove(os.path.join(label_path, l_dir))

In [ ]:
import cv2, os
from matplotlib import pyplot as plt
from pathlib import Path
path_to_image = Path("/home/th3will/dev/OTUMEDAI/training_data/yolo/images/train/test1_c1.jpg")
path_to_label = Path("/home/th3will/dev/OTUMEDAI/training_data/yolo/labels/train/test1_c1.txt")
img = cv2.imread(path_to_image.as_posix())
if(os.path.getsize(path_to_label.as_posix()) > 0):
    with open(path_to_label.as_posix()) as f:
        for line in f:
            line = line.split()
            print(line)
            x = float(line[1])
            y = float(line[2])
            w = float(line[3])
            h = float(line[4])
            x1 = int((x-w/2)*img.shape[1])
            y1 = int((y-h/2)*img.shape[0])
            x2 = int((x+w/2)*img.shape[1])
            y2 = int((y+h/2)*img.shape[0])
            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
plt.imshow(img)
cv2.waitKey(5000)
cv2.destroyAllWindows()


In [ ]:
# get every 10 frames and copy them to a new folder for segmentation
import shutil, os
from pathlib import Path
path_to_frames = '/home/th3will/dev/OTUMEDAI/training_data/yolo' 
new_dir = Path('/home/th3will/dev/OTUMEDAI/test')
if not os.path.exists(new_dir):
    os.mkdir(os.path.join(new_dir, "seg_source"))
i = 0
for file in os.listdir(os.path.join(path_to_frames, "images", "train")):
    if i %10 == 0:
        shutil.copy(os.path.join(path_to_frames, "images", "train", file), os.path.join(new_dir, "seg_source"))
    i+=1
